In [1]:
import os
import io
import matlab.engine
import sys
import copy
sys.path.append("../")
import od.vars as GV
import od.engine



In [2]:
print(matlab.engine.find_matlab())
print(os.getcwd())
MATLAB_ENG = matlab.engine.connect_matlab()
od.engine.MATLAB_ENG = MATLAB_ENG
MATLAB_ENG.addpath(os.getcwd() + "/../matlab/")
MATLAB_ENG.addpath(os.getcwd() + "/../matlab/SelectCQI_bySNR/")
MATLAB_ENG.addpath(os.getcwd() + "/../matlab/PlannerV1/")
print("Done")

('MATLAB_4034539',)
/home/OD/repos/ProjectSumo/ipynb
Done


In [3]:
from numpy import random

In [37]:
max_group_per_qos = 5
QoS_GP_CONF = [[
    {
        'gid': float(max_group_per_qos*x+i + 1),
        'qos': 1.0,
        **random.choice([{'rbf_w': 2.0, 'rbf_h': 1.0}, {'rbf_w': 1.0, 'rbf_h': 2.0}]),
        'sinr_max': float(random.randint(150, 300) / 10),
        **random.choice([
            {'pwr_req_dBm': 22.956, 'pwr_ext_dBm': 3.034702226027002},
            {'pwr_req_dBm': 22.959, 'pwr_ext_dBm': 2.729511581869497},
            {'pwr_req_dBm': 22.955, 'pwr_ext_dBm': 3.1318014527925526}
        ]),
        'rem_bits': float(random.randint(1000, 50000)),
        'mem_num': float(random.randint(3, 10)),
        'eager_rate': float(random.randint(1000, 50000) / 10)
    }
    for i in range(max_group_per_qos)
    ] for x in range(3)
]
RES_CONF = {'rbf_h': float(random.randint(1,5)*10) , 'rbf_w': 2.0, 'max_pwr_dBm': 23.0}

# QoS_GP_CONF = [
#     [
#         {
#             'gid': 5.0,
#             'qos': 1.0,
#             'rbf_w': 2.0,
#             'rbf_h': 1.0,
#             'sinr_max': 30.0,
#             'pwr_req_dBm': 22.996,
#             'pwr_ext_dBm': -7.3592430462207306,
#             'rem_bits': 4125123412,
#             'mem_num': 8.0,
#             'eager_rate': 3.445574916313482
#         },
#         {
#             'gid': 6.0,
#             'qos': 1.0,
#             'rbf_w': 1.0,
#             'rbf_h': 2.0,
#             'sinr_max': 30.0,
#             'pwr_req_dBm': 22.996,
#             'pwr_ext_dBm': -7.3592430462207306,
#             'rem_bits': 40978.0,
#             'mem_num': 8.0,
#             'eager_rate': 4.445574916313482
#         },
#         {
#             'gid': 7.0,
#             'qos': 1.0,
#             'rbf_w': 2.0,
#             'rbf_h': 1.0,
#             'sinr_max': 30.0,
#             'pwr_req_dBm': 22.996,
#             'pwr_ext_dBm': -7.3592430462207306,
#             'rem_bits': 12314413.0,
#             'mem_num': 8.0,
#             'eager_rate': 5.445574916313482
#         }
#     ]
# ]
# RES_CONF = {'rbf_h': 6 , 'rbf_w': 2.0, 'max_pwr_dBm': 23.0}
# QoS_GP_CONF = [[
#     {'gid': 1.0, 'qos': 1.0, 'rbf_w': 1.0, 'rbf_h': 2.0, 'sinr_max': 27.0, 'pwr_req_dBm': 22.955, 'pwr_ext_dBm': 3.1318014527925526, 'rem_bits': 46984.0, 'mem_num': 3.0, 'eager_rate': 2397.1}, 
#     {'gid': 2.0, 'qos': 1.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 19.0, 'pwr_req_dBm': 22.955, 'pwr_ext_dBm': 3.1318014527925526, 'rem_bits': 20810.0, 'mem_num': 8.0, 'eager_rate': 1533.1}, 
#     {'gid': 3.0, 'qos': 1.0, 'rbf_w': 1.0, 'rbf_h': 2.0, 'sinr_max': 21.3, 'pwr_req_dBm': 22.956, 'pwr_ext_dBm': 3.034702226027002, 'rem_bits': 33628.0, 'mem_num': 9.0, 'eager_rate': 1295.4}]]
# {'rbf_h': 20.0, 'rbf_w': 2.0, 'max_pwr_dBm': 23.0}
# RES_CONF = {'rbf_h': 20.0, 'rbf_w': 2.0, 'max_pwr_dBm': 23.0}
print(QoS_GP_CONF)
print(RES_CONF)


[[{'gid': 1.0, 'qos': 1.0, 'rbf_w': 1.0, 'rbf_h': 2.0, 'sinr_max': 19.5, 'pwr_req_dBm': 22.956, 'pwr_ext_dBm': 3.034702226027002, 'rem_bits': 8950.0, 'mem_num': 8.0, 'eager_rate': 4535.7}, {'gid': 2.0, 'qos': 1.0, 'rbf_w': 1.0, 'rbf_h': 2.0, 'sinr_max': 27.5, 'pwr_req_dBm': 22.956, 'pwr_ext_dBm': 3.034702226027002, 'rem_bits': 40370.0, 'mem_num': 6.0, 'eager_rate': 1270.9}, {'gid': 3.0, 'qos': 1.0, 'rbf_w': 1.0, 'rbf_h': 2.0, 'sinr_max': 26.1, 'pwr_req_dBm': 22.956, 'pwr_ext_dBm': 3.034702226027002, 'rem_bits': 47141.0, 'mem_num': 9.0, 'eager_rate': 859.2}, {'gid': 4.0, 'qos': 1.0, 'rbf_w': 2.0, 'rbf_h': 1.0, 'sinr_max': 26.5, 'pwr_req_dBm': 22.959, 'pwr_ext_dBm': 2.729511581869497, 'rem_bits': 15084.0, 'mem_num': 9.0, 'eager_rate': 4517.0}, {'gid': 5.0, 'qos': 1.0, 'rbf_w': 1.0, 'rbf_h': 2.0, 'sinr_max': 15.2, 'pwr_req_dBm': 22.956, 'pwr_ext_dBm': 3.034702226027002, 'rem_bits': 16718.0, 'mem_num': 8.0, 'eager_rate': 2040.3}], [{'gid': 6.0, 'qos': 1.0, 'rbf_w': 1.0, 'rbf_h': 2.0, 'sinr

In [38]:
out = io.StringIO()
try:
    alloc_report, exitflag = MATLAB_ENG.PlannerV1(RES_CONF, QoS_GP_CONF, nargout=2, stdout=out)
except Exception as e:
    raise e
print(out)
for gid in sorted(alloc_report.keys()):
    print(gid,alloc_report[gid])

g1 {'t0': {}, 't1': {}}
g10 {'t0': {'c13': 14}, 't1': {'c13': 14}}
g11 {'t0': {}}
g12 {'t0': {}}
g13 {'t0': {}}
g14 {'t0': {}, 't1': {}}
g15 {'t0': {}}
g2 {'t0': {}, 't1': {}}
g3 {'t0': {}, 't1': {}}
g4 {'t0': {'c8': 40}}
g5 {'t0': {}, 't1': {}}
g6 {'t0': {}, 't1': {}}
g7 {'t0': {}}
g8 {'t0': {}}
g9 {'t0': {'c15': 12}}


In [39]:
import importlib
importlib.reload(od.network.allocator)
from od.network.allocator import ResourceAllocatorNomaApprox

In [40]:
allocator = ResourceAllocatorNomaApprox(copy.deepcopy(RES_CONF),copy.deepcopy(QoS_GP_CONF))
alloc_report = allocator()
for gid in sorted(alloc_report.keys()):
    print(gid,alloc_report[gid])

g1 {'t0': {'c14': 11}}
g2 {'t0': {'c8': 19}, 't1': {'c15': 11, 'c8': 8}}
g4 {'t0': {'c15': 17}}
g5 {'t1': {'c2': 11}}
g7 {'t0': {'c2': 1}}
g9 {'t0': {'c15': 1, 'c6': 1}}


In [41]:
print(allocator.spare_inner_rb)
print(allocator.solid_normal_rb)
print(allocator.spare_hybrid_rb)
print(allocator.solid_hybrid_rb)

{2: {1: []}, 1: {2: []}}
{2: {1: [{I:(g1:c14),O:(g2:c8)}, {I:(g1:c14),O:(g2:c8)}, {I:(g1:c14),O:(g2:c8)}, {I:(g1:c14),O:(g2:c8)}, {I:(g1:c14),O:(g2:c8)}, {I:(g1:c14),O:(g2:c8)}, {I:(g1:c14),O:(g2:c8)}, {I:(g1:c14),O:(g2:c8)}, {I:(g1:c14),O:(g2:c8)}, {I:(g1:c14),O:(g2:c8)}, {I:(g1:c14),O:(g2:c8)}, {I:(g2:c15),O:(g5:c2)}, {I:(g2:c15),O:(g5:c2)}, {I:(g2:c15),O:(g5:c2)}, {I:(g2:c15),O:(g5:c2)}, {I:(g2:c15),O:(g5:c2)}, {I:(g2:c15),O:(g5:c2)}, {I:(g2:c15),O:(g5:c2)}, {I:(g2:c15),O:(g5:c2)}, {I:(g2:c15),O:(g5:c2)}, {I:(g2:c15),O:(g5:c2)}, {I:(g2:c15),O:(g5:c2)}]}, 1: {2: [{I:(g4:c15),O:(g9:c6)}, {I:(g9:c15),O:(g7:c2)}]}}
{2: {1: []}, 1: {2: []}}
{2: {1: []}, 1: {2: [{I:[(g4:c15), (g4:c15)],O:[(g2:c8), (g2:c8)]}, {I:[(g4:c15), (g4:c15)],O:[(g2:c8), (g2:c8)]}, {I:[(g4:c15), (g4:c15)],O:[(g2:c8), (g2:c8)]}, {I:[(g4:c15), (g4:c15)],O:[(g2:c8), (g2:c8)]}, {I:[(g4:c15), (g4:c15)],O:[(g2:c8), (g2:c8)]}, {I:[(g4:c15), (g4:c15)],O:[(g2:c8), (g2:c8)]}, {I:[(g4:c15), (g4:c15)],O:[(g2:c8), (g2:c8)]}, {I:

In [180]:
import math
_SINR = 10**(3.1318014527925526/10) * (10**(19.0/10)/ 10**(23/10))
SINR = 10 * math.log10(_SINR)
print(_SINR,SINR)
MATLAB_ENG.SelectCQI_BLER10P(SINR)

0.8188043576153365 -0.8681985472074462


4.0

In [20]:
for conf in QoS_GP_CONF[0]:
    print(MATLAB_ENG.SelectCQI_BLER10P(conf["sinr_max"]) * conf["eager_rate"])

35956.5
21463.399999999998
19431.0
